#### Функция токенизации

Реализуем функцию `tokenize`, преобразующую входную строку в список токенов на основе предопределённых паттернов. Токены представляют собой минимальные смысловые единицы, такие как переменные, лямбда-символы, скобки и т.д.


In [1]:
def tokenize(input):
    tokens = []
    i = 0
    while i < len(input):
        if input[i].isspace():
            i += 1
        elif input[i] == '\\':
            tokens.append('LAMBDA')
            i += 1
        elif input[i] == 'λ':
            tokens.append('LAMBDA')
            i += 1
        elif input[i] == '.':
            tokens.append('DOT')
            i += 1
        elif input[i] == '(':
            tokens.append('LPAREN')
            i += 1
        elif input[i] == ')':
            tokens.append('RPAREN')
            i += 1
        elif input[i].isalpha():
            tokens.append(('VAR', input[i]))
            i += 1
        else:
            raise Exception(f"Unknown character: {input[i]} at position {i}")
    return tokens


#### Тестирование токенизации
Определяем тестовое лямбда-выражение и применяем функцию токенизации для получения списка токенов.

In [2]:
lambda_expr = "λx.(λy.(y x) x λxy.xy)x"
tokens = tokenize(lambda_expr)
print(tokens)

['LAMBDA', ('VAR', 'x'), 'DOT', 'LPAREN', 'LAMBDA', ('VAR', 'y'), 'DOT', 'LPAREN', ('VAR', 'y'), ('VAR', 'x'), 'RPAREN', ('VAR', 'x'), 'LAMBDA', ('VAR', 'x'), ('VAR', 'y'), 'DOT', ('VAR', 'x'), ('VAR', 'y'), 'RPAREN', ('VAR', 'x')]


### Определения классов

В этой части кода определяются основные классы, которые будут использоваться для представления узлов абстрактного синтаксического дерева (АСТ) лямбда-выражений


In [3]:
class LambdaNode:
    def __init__(self, variables, body):
        self.variables = variables  # Список переменных
        self.body = body  # Тело функции (список токенов или LambdaNode)
        
    def __repr__(self):
        return self.pretty_print()
        
    def pretty_print(self, indent=0):
        result = ''
        if (len(self.variables) > 0):
            result += ' ' * indent + 'LAMBDA ' + ' '.join(self.variables) + ' .\n'
        for node in self.body:
            if isinstance(node, LambdaNode):
                result += node.pretty_print(indent + 4)
            else:
                result += ' ' * (indent + 4) + str(node) + '\n'
        return result

#### Парсер

Эта часть содержит реализацию парсера, который преобразует последовательность токенов, в АСТ. Парсер следует грамматике лямбда-выражений для построения дерева, корректно определяя структуру вложенных функций и применений.


In [4]:
def parse_lambda(tokens, start = 0, end = None):
    variables = []
    body = []
    if end is None:
        end = len(tokens)

    if tokens[start] == 'LAMBDA':
        # Собираем переменные функции
        start += 1  # Пропускаем токен 'LAMBDA'
        while tokens[start] != 'DOT':
            if tokens[start][0] == 'VAR':
                variables.append(tokens[start][1])
            start += 1
    
        start += 1  # Пропускаем токен 'DOT'
        
    # Обрабатываем тело функции
    while start < end:
        if tokens[start] == 'LPAREN':
            #body.append(tokens[start])
            i = start
            balance = 1
            while balance != 0:
                assert i < end, "Unbalanced parentheses"
                i += 1
                if tokens[i] == 'LPAREN':
                    balance += 1
                elif tokens[i] == 'RPAREN':
                    balance -= 1
            body.append(parse_lambda(tokens, start+1, i))
            #body.append(tokens[i])
            start = i + 1
        elif tokens[start] == 'LAMBDA':
            # Рекурсивный разбор вложенной лямбды
            node = parse_lambda(tokens, start, end+1)
            body.append(node)
            start += len(node.variables) + 2 + len(node.body)
        else:
            if tokens[start][0] == 'VAR':
                body.append(tokens[start])
            start += 1

    return LambdaNode(variables, body)



#### Визуализация AST
Парсим тестовый набор токенов, строим AST и выводим его структуру для визуальной проверки корректности парсинга.

In [5]:
ast = parse_lambda(tokens)
print("Lambda:", lambda_expr)
print(ast)

Lambda: λx.(λy.(y x) x λxy.xy)x
LAMBDA x .
    LAMBDA y .
            ('VAR', 'y')
            ('VAR', 'x')
        ('VAR', 'x')
        LAMBDA x y .
            ('VAR', 'x')
            ('VAR', 'y')
    ('VAR', 'x')


#### Альфа-конверсия

В данном блоке кода реализована функция альфа-конверсии. Альфа-конверсия необходима для переименования переменных в лямбда-выражениях, чтобы избежать конфликтов имен в различных областях видимости.


In [6]:
def generate_unique_var_name(var_name, existing_vars):
    """Генерирует уникальное имя переменной на основе существующих."""
    unique_var_name = var_name
    count = 1
    while unique_var_name in existing_vars:
        unique_var_name = f"{var_name}_{count}"
        count += 1
    return unique_var_name

def alpha_conversion(node, parent_vars=set()):
    # Проверяем наличие переменных с одинаковыми именами
    new_variables_dict = {}
    for var in node.variables:
        if var in parent_vars:
            new_var = generate_unique_var_name(var, parent_vars)
            new_variables_dict[var] = new_var
            parent_vars.add(new_var)
        else:
            parent_vars.add(var)
            
    def replace_var(node, var_dict):
        if isinstance(node, LambdaNode):
            node.variables = [var_dict.get(var, var) for var in node.variables]
            for i in range(len(node.body)):
                if isinstance(node.body[i], tuple) and node.body[i][0] == 'VAR':
                    node.body[i] = ('VAR', var_dict.get(node.body[i][1], node.body[i][1]))
                else:
                    replace_var(node.body[i], var_dict)
    
    replace_var(node, new_variables_dict)
    
    # Рекурсивно вызываем alpha_conversion для всех дочерних узлов
    for i in range(len(node.body)):
        if isinstance(node.body[i], LambdaNode):
            alpha_conversion(node.body[i], parent_vars)
           
           

#### Тестирование альфа-конверсии
Применяем альфа-конверсию к тестовому AST и выводим результат. Получаем сет переменных, которые были переименованы.

In [7]:
vars_set = set()
alpha_conversion(ast, vars_set)

print("Lambda:", lambda_expr)
print("Renamed variables set:", vars_set)
print(ast)

Lambda: λx.(λy.(y x) x λxy.xy)x
Renamed variables set: {'y_1', 'y', 'x', 'x_1'}
LAMBDA x .
    LAMBDA y .
            ('VAR', 'y')
            ('VAR', 'x')
        ('VAR', 'x')
        LAMBDA x_1 y_1 .
            ('VAR', 'x_1')
            ('VAR', 'y_1')
    ('VAR', 'x')


#### Бета-редукция

Эти ячейки содержат реализацию бета-редукции, процесса вычисления (или упрощения) лямбда-выражений путем подстановки аргументов функции вместо соответствующих параметров. Бета-редукция является ключевой операцией в лямбда-исчислении, позволяющей "выполнить" лямбда-выражение.


In [8]:
def beta_reduction(node, values):
    if isinstance(node, LambdaNode):
        node.variables = ["(" + var + " = " + str(values.get(var, var)) + ")" for var in node.variables]
        #node.variables = [str(values.get(var, var)) for var in node.variables]
        for i in range(len(node.body)):
            if isinstance(node.body[i], tuple) and node.body[i][0] == 'VAR':
                if node.body[i][1] in values:
                    #node.body[i] = ('VAR', values[node.body[i][1]])
                    node.body[i] = "(" + node.body[i][1] + " = " + str(values[node.body[i][1]]) + ")"
            else:
                beta_reduction(node.body[i], values)

#### Тестирование бета-редукции
Применяем бета-редукцию к AST, используя словарь значений переменных. Визуализируем результат.

In [9]:
values = {'x' : 1, 'y' : 2, 'x_1' : 3, 'y_1' : 4}
beta_reduction(ast, values)
print(ast)

LAMBDA (x = 1) .
    LAMBDA (y = 2) .
            (y = 2)
            (x = 1)
        (x = 1)
        LAMBDA (x_1 = 3) (y_1 = 4) .
            (x_1 = 3)
            (y_1 = 4)
    (x = 1)
